# Road Following Data Editing - Adding xy Coordinates to Photos by Mouseclick

Author: Travis Moore, NNMC STARR Team

Note: This code draws heavily from three sources. First, the NVIDIA data_collection.ipynb notebook shipped with the Jetbot sold by Sparkfun.com. You can find the original code here: https://github.com/NVIDIA-AI-IOT/jetbot

Second, George Gorospe's modified data collection notebook that uses the Jupyter Clickable Widgets library to select a best path by mouseclick rather than scrollbars (https://github.com/jaybdub/jupyter_clickable_image_widget). 

Third, the BalticBot Poland STARR team's fantastic notebook that provides functionality to mark X,Y coordinates on existing images.

In [ ]:
# Assign function to save button
saveButton.on_click(lambda x: next_photo())

In [21]:
# Import libraries
# IPython Libraries for display and widgets
import ipywidgets.widgets as widgets
import traitlets
from IPython.display import display
from ipywidgets import interact, interact_manual
from jupyter_clickable_image_widget import ClickableImageWidget

# Python basic packages for image annotation
import os
import numpy as np
import cv2
import time

# Create a list of photos from an existing directory
DIR = 'dataset_test/free'
photo_list = os.listdir(DIR)
nr_photos = len(photo_list)
print("You have %d photos in that folder." % nr_photos)

# Select the first photo from the list
file = photo_list[0]
print(file)

# Define the first image from the list
img = cv2.imread(DIR+'/'+file)
# Encode the first image as a jpeg
imv = cv2.imencode('.jpg', img)[1].tobytes()

# Create a placeholder image to identify the end of the dataset
end_img = np.zeros((224,224,3), np.uint8)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(end_img,'END OF DATASET',(60,120), font, 0.4,(255,255,255),1,cv2.LINE_AA)
imv_end = cv2.imencode('.jpg', end_img)[1].tobytes()
image_widget.value = imv_end

# Create a widget to display the current image
image_widget = ClickableImageWidget(value = imv, format='jpeg', width=224, height=224)
display(image_widget)

# Create save button
button_layout = widgets.Layout(width='128px', height='64px', align_self='center')
saveButton = widgets.Button(description='SAVE', button_style='success', layout=button_layout)

# Create a new directory to save the modified images
DATASET_XY_DIR = 'new_dataset_test'
try:
    os.makedirs(DATASET_XY_DIR)
except FileExistsError:
    print('Directory not created because it already exists')

# Apply the x and y offset data to the current image's title and save in the new directory
def save_photo(content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        file = photo_list[count]
    
        new_file = 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, file)
        
        image_path = os.path.join(DATASET_XY_DIR, new_file)
        with open(image_path, 'wb') as f: 
            f.write(imv)
            
        # Display new snapshot with Circle marking
        snapshot = np.copy(img)
        snapshot = cv2.circle(snapshot, (x,y), 8, (0,255,0), 3)
        imv=cv2.imencode('.jpg', snapshot)[1].tobytes()
        image_widget.value = imv
        
display(image_widget)

# Function to save the current image and move on to the next one
def next_photo():
    global count, xd, yd, xs, ys
    save_photo()
    count = (count + 1)
    if count < len(photo_list):
        file = photo_list[count]
        img = cv2.imread(DIR + '/' + file)
        img = cv2.circle(img, (xd,yd), 8 (0,255,0), 3)
        imv = cv2.imencode('.jpg', img)[1].tobytes()
        image_widget.value = imv
    else:
        image_widget.value = imv_end
        
count = 0
@interact
def show_images():
    global count, xd, yd, xs, ys
    if count < len(photo_list):
        file = photo_list[count]
        image = np.copy(img)
        if content['event'] == 'click':
            data = content['eventData']
            x = data['offsetX']
            y = data['offsetY']
        xs = x
        ys = y
        image = cv2.circle(image, (x,y), 8, (0,255,0), 3)
        image_widget.value = imv
        xd = x
        yd = y

display(widgets.Hbox([image_widget, saveButton]))

You have 1211 photos in that folder.
xy_046_046_9952c804-7df0-11eb-add8-08beac06b6d8.jpg


ClickableImageWidget(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C…

Directory not created because it already exists


ClickableImageWidget(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C…

interactive(children=(Output(),), _dom_classes=('widget-interact',))

AttributeError: module 'ipywidgets.widgets' has no attribute 'Hbox'